In [1]:
import os
os.chdir("/data/npl/ViInfographicCaps/Contest/final_contest/XAI")
from utils import load_json
import re
from collections import defaultdict

/home/npl/.conda/envs/diffsinger/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
json_file = load_json("/data/npl/ViInfographicCaps/Contest/final_contest/XAI/save/paraphrase.json")

In [ ]:
print()

['\n            <s>[INST] <<SYS>>\n            ### Instruction:\n            \n        \n        You are an AI reasoning assistant. Your task is to understand and answer questions based on events and rules that happen in an educational setting. The dataset contains simple, rule-based statements (like course completions, enrollments, and eligibility) and facts about individuals (like students).\n        Your job is to:\n        - The event must include both verb and object related to that event (submit a form → submit_form)\n        - Identify logical relationships between events (e.g., "If A, then B"),\n        - Track facts about entities (like what courses David completed),\n        - Infer new facts by chaining rules together (e.g., if David completed_A and passed_B, then he may be David eligible_for_something),\n        - Keep all names and objects consistent (do not rename),\n        Avoid subject-specific knowledge—the task is about reasoning in an educational context, not teachi

In [ ]:
def get_paraphrase_info(response):
    text = response.split("### Answer Response:")[-1]
    pattern_statements = r"Simplified Statement \d+: .*$"
    pattern_objects = r"List Objects \d+: .*$"
    pattern_actions = r"List Actions \d+: .*$"
    pattern_instances = r"List Instances \d+: .*$"
    pattern_information = r"Importance Information \d+: .*$"

    matches_statements = re.findall(pattern_statements, text, re.MULTILINE)
    matches_objects = re.findall(pattern_objects, text, re.MULTILINE)
    matches_actions = re.findall(pattern_actions, text, re.MULTILINE)
    matches_instances = re.findall(pattern_instances, text, re.MULTILINE)
    matches_information = re.findall(pattern_information, text, re.MULTILINE)
    
    return matches_statements, matches_objects, matches_actions, matches_instances, matches_information

In [25]:
def parse_info(info, sep=','):
    '''
        Sep to split into array
    '''
    match = re.search(r':\s*(.*)', info).group(1)
    if sep != None:
        items = match.split(sep)
        # postprocess for list instances
        items = [None if item=="No Instance" else item.strip() for item in items]
        return items
    return match

In [26]:
response = json_file['0']
matches_statements, matches_objects, matches_actions, matches_instances, matches_information = get_paraphrase_info(response=response)
category_names = ['simplyfied statement', 'list objects', 'list actions', 'list instances', 'list information']
categories = [matches_statements, matches_objects, matches_actions, matches_instances, matches_information]

dic_info = {}
for cat_name, cat in zip (category_names, categories):
    if cat_name in ['list objects', 'list actions', 'list instances']:
        split = ","
    elif cat_name == "list information":
        split = '.'
    else:
        split = None
    parsed_cat = [parse_info(cat_content, split) for cat_content in cat]
    dic_info[cat_name] = parsed_cat
        

In [27]:
dic_info

{'simplyfied statement': ['A student can withdraw up to 3 courses/academic year if credits ≥ 10 (Year 1, 2) or ≥ 8 (Year 3+); each withdrawal deducts 0.5 credits from total accumulated credits.',
  'Courses have credits; withdrawn courses contribute 0 credits to semester GPA.',
  'Students must register for credits/semester; withdrawals reduce semester credits.',
  'No regulation limits total withdrawals; max 3/year.',
  'A student with < 8 accumulated credits cannot withdraw courses.',
  'A student has credits; withdrew courses in Year 1 (penalty credit); C1 (4 credits, withdrawn), C2 (3 credits, withdrawn).'],
 'list objects': [['courses', 'academic year', 'credits'],
  ['courses', 'credits'],
  ['credits', 'semester'],
  ['withdrawals'],
  ['credits'],
  ['credits', 'courses']],
 'list actions': [['can withdraw', 'deduct'],
  ['have', 'contribute'],
  ['must register', 'reduce'],
  ['limit', 'max'],
  ['cannot withdraw'],
  ['has', 'withdrew', 'penalty']],
 'list instances': [['3', 

In [ ]:
# def clean(text):
#     cleaned_text = text.split("$$$$")[-1]
#     elements = defaultdict(set)

#     statements = {}

#     # Regex patterns for identifying each section
#     stmt_pattern = re.compile(r'^Simplified Statement (\d+): (.+)$')
#     list_pattern = re.compile(r'^List (Objects|Actions|Instances) (\d+):\s*(.*)$')

#     current_stmt = None

#     for line in cleaned_text.splitlines():
#         line = line.strip()
#         if not line:
#             continue

#         # Match statement
#         stmt_match = stmt_pattern.match(line)
#         if stmt_match:
#             stmt_num, stmt_text = stmt_match.groups()
#             current_stmt = f"Statement {stmt_num}"
#             statements[current_stmt] = {
#                 "text": stmt_text,
#                 "objects": [],
#                 "actions": [],
#                 "instances": []
#             }
#             continue

#         # Match associated objects/actions/instances
#         list_match = list_pattern.match(line)
#         if list_match and current_stmt:
#             category, num, items = list_match.groups()
#             if f"Statement {num}" == current_stmt:
#                 elements = [item.strip() for item in items.split(",") if item.strip() and item.lower() != "no instance"]
#                 category_key = category.lower()
#                 statements[current_stmt][category_key + 's'] = elements

#     return statements
# clean(json_file['0'])